# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "output.csv"
csv = "output.csv"
weather_data_df = pd.read_csv("output.csv")
weather_data_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,altagracia de orituco,9.8601,-66.3814,71.35,84,49,7.02,VE,1615508315
1,svetlaya,46.5389,138.3297,32.72,60,5,8.88,RU,1615508410
2,new norfolk,-42.7826,147.0587,68.47,54,12,2.84,AU,1615508410
3,butaritari,3.0707,172.7902,81.61,75,56,13.73,KI,1615508410
4,juneau,58.3019,-134.4197,29.43,64,90,10.36,US,1615508411
...,...,...,...,...,...,...,...,...,...
580,kidal,18.4411,1.4078,70.52,14,17,13.29,ML,1615508592
581,newport,51.5877,-2.9984,43.74,80,20,1.01,GB,1615508593
582,eydhafushi,5.1033,73.0708,82.92,74,100,18.52,MV,1615508593
583,lai,9.3952,16.2989,83.08,21,95,1.23,TD,1615508593


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

coordinates =weather_data_df[["lat", "long"]]
humidity = weather_data_df["humidity"].astype(float)

fig = gmaps.figure()
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom' : '8x'
}

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_df = weather_data_df
# Create the names and bins for temperature
temp_bins = [-100,70, 80, 400]
temp_group_names = ["too cool", "70-80", "too hot"]

# Create the names and bins for wind
wind_bins = [-5, 10, 500]
wind_group_names = ["less than 10 mph", "more than 10 mph"]

# Create the names and bins for cloudiness
cloud_bins = [-5, 0.1, 500]
cloud_group_names = ["clear", "cloudy"]


vacation_df["temp"] = pd.cut(vacation_df["temp"], temp_bins, labels=temp_group_names, include_lowest=True)
vacation_df["wind"] = pd.cut(vacation_df["wind"], wind_bins, labels=wind_group_names, include_lowest=True)
vacation_df["cloud"] = pd.cut(vacation_df["cloud"], cloud_bins, labels=cloud_group_names, include_lowest=True)
vacation_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
0,altagracia de orituco,9.8601,-66.3814,70-80,84,cloudy,less than 10 mph,VE,1615508315
1,svetlaya,46.5389,138.3297,too cool,60,cloudy,less than 10 mph,RU,1615508410
2,new norfolk,-42.7826,147.0587,too cool,54,cloudy,less than 10 mph,AU,1615508410
3,butaritari,3.0707,172.7902,too hot,75,cloudy,more than 10 mph,KI,1615508410
4,juneau,58.3019,-134.4197,too cool,64,cloudy,more than 10 mph,US,1615508411
...,...,...,...,...,...,...,...,...,...
580,kidal,18.4411,1.4078,70-80,14,cloudy,more than 10 mph,ML,1615508592
581,newport,51.5877,-2.9984,too cool,80,cloudy,less than 10 mph,GB,1615508593
582,eydhafushi,5.1033,73.0708,too hot,74,cloudy,more than 10 mph,MV,1615508593
583,lai,9.3952,16.2989,too hot,21,cloudy,less than 10 mph,TD,1615508593


In [5]:
ideal_vacation_df=vacation_df.loc[vacation_df["temp"]=="70-80"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["wind"]=="less than 10 mph"]
ideal_vacation_df=ideal_vacation_df.loc[ideal_vacation_df["cloud"]=="clear"]
hotel_df =ideal_vacation_df.loc[ideal_vacation_df["country"]=="IN"].copy()
hotel_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time
32,mangrol,21.1167,70.1167,70-80,80,clear,less than 10 mph,IN,1615508420
291,dwarka,22.2394,68.9678,70-80,79,clear,less than 10 mph,IN,1615508502
351,shirdi,19.7667,74.4833,70-80,30,clear,less than 10 mph,IN,1615508520
505,veraval,20.9000,70.3667,70-80,81,clear,less than 10 mph,IN,1615508568
535,port blair,11.6667,92.7500,70-80,79,clear,less than 10 mph,IN,1615508578


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel = []
locations = hotel_df["city"]
# geocoordinates

target_radius = 5000
target_type = "lodging"

for index,row in hotel_df.iterrows():
    target_coordinates = f'{row["lat"]},{row["long"]}'
# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary

    
    response = requests.get(base_url, params=params)
#hotel.append(response['hotel'])
    #print(response.json())
    hotel_response=response.json()
    hotel.append(hotel_response["results"][0]["name"])
hotel_df["Hotel Name"]=""
hotel_df.loc[:,"Hotel Name"]=hotel

hotel_df

,city,lat,long,temp,humidity,cloud,wind,country,date and time,Hotel Name
32,mangrol,21.1167,70.1167,70-80,80,clear,less than 10 mph,IN,1615508420,Hotel The Royal Honours
291,dwarka,22.2394,68.9678,70-80,79,clear,less than 10 mph,IN,1615508502,Hotel Narayan Avezika Comfort
351,shirdi,19.7667,74.4833,70-80,30,clear,less than 10 mph,IN,1615508520,Shradha Saburi Palace
505,veraval,20.9000,70.3667,70-80,81,clear,less than 10 mph,IN,1615508568,Lords Inn Somnath
535,port blair,11.6667,92.7500,70-80,79,clear,less than 10 mph,IN,1615508578,Welcomhotel Bay Island Port Blair


In [7]:
hotel_df=hotel_df.rename(columns={"city":"City", "country":"Country", "lat":"Lat", "long":"Lng"})


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig
                             
# Display figure


Figure(layout=FigureLayout(height='420px'))